In [1]:
!pip install transformers
!pip install emoji
# !pip install cloud-tpu-client==0.10 torch==1.10.0 https://storage.googleapis.com/tpu-pytorch/wheels/colab/torch_xla-1.11-cp37-cp37m-linux_x86_64.whl

In [2]:
import re
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer, AutoModel, BertConfig
import pandas as pd
from tqdm import tqdm

# import torch_xla
# import torch_xla.core.xla_model as xm
# import torch_xla.distributed.parallel_loader as pl
# import torch_xla.distributed.xla_multiprocessing as xmp
# import torch_xla.utils.utils as xu

In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
torch.cuda.empty_cache()
!nvidia-smi

Tue Apr 26 12:42:22 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   71C    P8    14W /  70W |      3MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [4]:
bert_model = "vinai/bertweet-base"
# bert_model = 'bert-base-uncased'

In [5]:
class TweetDataset(Dataset):

    def __init__(self, path, bert_model=bert_model, is_test=False):

        self.df = pd.read_csv(path, delimiter = '\t')
        self.tokenizer = AutoTokenizer.from_pretrained(bert_model)
        self.is_test = is_test

    def __len__(self):
        return len(self.df)

    def __getitem__(self, index):
        
        tweets = self.df.loc[index, 'text']
        
        tweets = self.preprocess(tweets)
        inputs = self.tokenizer(tweets, padding='max_length', truncation=True, return_tensors="pt")
        
        input_ids = inputs['input_ids'][0]
        attention_mask = inputs['attention_mask'][0]

        if not self.is_test:
            return input_ids, attention_mask, self.df.loc[index, 'label']
        else:
            return input_ids, attention_mask
    
    def preprocess(self, text):
        # text = text.replace('\n', '</s>')
        # text = re.sub(r'https?://t.co/[a-zA-Z0-9]+', '', text)
        
        return text

In [11]:
class RumourDetector(nn.Module):
    def __init__(self, bert_model=bert_model):
        super(RumourDetector, self).__init__()
        self.bert_block = AutoModel.from_pretrained(bert_model)

        self.clf_block = nn.Sequential(
            nn.Dropout(),
            nn.Linear(BertConfig.from_pretrained(bert_model).hidden_size, 128),
            nn.ReLU(),
            nn.Dropout(),
            nn.Linear(128, 1),
            nn.Sigmoid(),
        )

        # for p in self.bert_block.parameters():
        #     p.requires_grad = False

    def forward(self, input, attn_mask):
        outputs = self.bert_block(input, attention_mask=attn_mask)

        cls_rep = outputs.last_hidden_state[:, 0, :]

        probs = self.clf_block(cls_rep)
        preds = (probs > 0.5).int()

        return probs.flatten(), preds.flatten()


In [7]:
def train(model, optim, epoch_size, train_loader, valid_loader):
    loss_fn = nn.BCELoss()
    train_status = {'train_loss': [], 'train_acc': [], 'valid_acc': [], 
                    'checkpoint': {}}
    max_valid_acc = 0
    for epoch in range(epoch_size):
        model.train()
        epoch_loss = 0
        epoch_acc = 0
        train_loop = tqdm(enumerate(train_loader), total=len(train_loader))
        train_loop.set_description(f"Epoch [{epoch+1}/{epoch_size}]")

        for batch, (inputs, attention_masks, labels) in train_loop:
            inputs = inputs.to(device)
            attention_masks = attention_masks.to(device)
            labels = labels.float().to(device)
            probs, preds = model(inputs, attention_masks)
            loss = F.binary_cross_entropy(probs, labels)

            optim.zero_grad()
            loss.backward()
            optim.step() 

            epoch_loss += loss.item()
            epoch_acc += (preds == labels).float().mean().item()
            train_loop.set_postfix_str(
                'train_loss={:.3f}, train_acc={:.3f}'.format(
                    epoch_loss/(batch+1), epoch_acc/(batch+1)
                )
            )

            del inputs, attention_masks, labels
        
            if batch == len(train_loader)-1:
                valid_acc = validate(model, valid_loader)
                if valid_acc > max_valid_acc:
                    max_valid_acc = valid_acc
                    train_status['checkpoint']['state_dict'] = model.state_dict()
                train_status['valid_acc'].append(valid_acc)
                train_status['train_loss'].append(epoch_loss/(batch+1))
                train_status['train_acc'].append(epoch_acc/(batch+1))
                train_loop.set_postfix_str(
                    'train_loss={:.3f}, train_acc={:.3f}, valid_acc={:.3f}'.format(
                        train_status['train_loss'][-1],
                        train_status['train_acc'][-1],
                        train_status['valid_acc'][-1]
                    )
                )

    train_status['checkpoint']['train_status'] = train_status
    return train_status

def validate(model, valid_loader):
    model.eval()
    acc = 0
    for batch, (inputs, attention_masks, labels) in enumerate(valid_loader):
        inputs = inputs.to(device)
        attention_masks = attention_masks.to(device)
        labels = labels.float().to(device)
        _, preds = model(inputs, attention_masks)
        acc += (preds == labels).float().mean()
        del inputs, attention_masks, labels
    return acc / len(valid_loader)

In [8]:
from google.colab import drive
drive.mount('/content/gdrive/') 

Drive already mounted at /content/gdrive/; to attempt to forcibly remount, call drive.mount("/content/gdrive/", force_remount=True).


In [12]:
epoch_size = 10
batch_size = 16
lr = 1e-5

train_set = TweetDataset('/content/gdrive/MyDrive/data/train.csv')
train_loader = torch.utils.data.DataLoader(train_set, batch_size=batch_size, 
                                           shuffle=True, num_workers=0)

valid_set = TweetDataset('/content/gdrive/MyDrive/data/dev.csv')
valid_loader = torch.utils.data.DataLoader(valid_set, batch_size=batch_size, 
                                           shuffle=True, num_workers=0)

test_set = TweetDataset('/content/gdrive/MyDrive/data/test.csv', is_test=True)
test_loader = torch.utils.data.DataLoader(test_set, batch_size=1, 
                                           shuffle=False, num_workers=0)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [13]:
model = RumourDetector().to(device)
optim = torch.optim.Adam(model.parameters(), lr=lr)
train_status = train(model, optim, epoch_size, train_loader, valid_loader)

Some weights of the model checkpoint at vinai/bertweet-base were not used when initializing RobertaModel: ['lm_head.layer_norm.bias', 'lm_head.dense.bias', 'lm_head.decoder.weight', 'lm_head.bias', 'lm_head.decoder.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
You are using a model of type roberta to instantiate a model of type bert. This is not supported for all configurations of models and can yield errors.
Epoch [10/10]: 100%|██████████| 113/113 [00:49<00:00,  2.31it/s, train_loss=0.120, train_acc=0.972, valid_acc=0.89

In [14]:
torch.save(train_status['checkpoint'], '/content/gdrive/MyDrive/model/tweet_clf.pt')

In [15]:
def test(model, test_loader):
    model.eval()
    labels = []
    for batch, (inputs, attention_masks) in enumerate(test_loader):
        inputs = inputs.to(device)
        attention_masks = attention_masks.to(device)
        _, preds = model(inputs, attention_masks)
        preds = preds.tolist()
        labels.extend(preds)
        del inputs, attention_masks
    df = pd.DataFrame({'Id': list(range(0, len(test_loader))), 'Predicted': labels})
    df.to_csv('/content/gdrive/MyDrive/data/test.pred.csv', sep=',', index=False, encoding='utf-8')

In [16]:
test(model, test_loader=test_loader)